# Bayesian Optimization

@author:<tb> Kareem<br>
@date: <tb>28.04.2019
    
In the following I'd like to domenstrate the Bayesian Optimization process, in order for us to better handle continous values during model optimization. This implementaion relies on this [GitHub package](https://github.com/fmfn/BayesianOptimization). So to get started of course:

```
pip install bayesian-optimization
```

## Loading Data

Let's load some data from sklearn default datasets and then we will build different classifiers to evaluate how well this implemntation is

In [1]:
#### Load Iris Data ####
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset['data']
y = dataset['target']

## Split into train and validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

## Handling Continous Values

There's immediate support for this type of parameters from the package, so let's build a logistic regressor right away and see the accuracies we can get.

Note that the object `BayesianOptimization`, always needs the objective function (i.e. function to be maximized) as constructor argument. If you want to minimize a function, just return the minus value of that function.

In [2]:
#### Build Classifier ####
from sklearn.linear_model import LogisticRegression
def black_box_function(C):
    """Function with we wish to maximize."""
    classifier = LogisticRegression(C=C, random_state=0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test) # irrelevent!
    return classifier.score(X_test, y_test) # return accuracy

`C` will take all the values range from $0$ to $1$.

In [4]:
from bayes_opt import BayesianOptimization

In [11]:
#### Build the Optimization Space ####
from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'C': (0, 1)} 

optimizer = BayesianOptimization(
    f=black_box_function, # function to optimize
    pbounds=pbounds, # dimension to explore
    random_state=0)
optimizer.maximize(
    init_points=5, # number of exploration steps
    n_iter=10) # number of exploitation steps

print(optimizer.max) # best score and parameter set
for i, res in enumerate(optimizer.res): # full optimization history
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.9561   |  0.5488   |
|  2        |  0.9561   |  0.7152   |
|  3        |  0.9561   |  0.6028   |
|  4        |  0.9561   |  0.5449   |
|  5        |  0.9561   |  0.4237   |
|  6        |  0.9386   |  0.02578  |
|  7        |  0.9561   |  1.0      |
|  8        |  0.9561   |  1.0      |
|  9        |  0.9561   |  0.8043   |
|  10       |  0.9561   |  0.7758   |
|  11       |  0.9561   |  1.0      |
|  12       |  0.9561   |  0.6563   |
|  13       |  0.9561   |  1.0      |
|  14       |  0.9561   |  0.8092   |
|  15       |  0.9561   |  0.5683   |
{'params': {'C': 0.5488135039273248}, 'target': 0.956140350877193}
Iteration 0: 
	{'params': {'C': 0.5488135039273248}, 'target': 0.956140350877193}
Iteration 1: 
	{'params': {'C': 0.7151893663724195}, 'target': 0.956140350877193}
Iteration 2: 
	{'params': {'C': 0.6027633760716439}, 'target': 0.956140350877193}
Iteration 3: 
	{'params': {'C': 0.5448831

## Handling Discrete Values

Discrete values are not handled by the package and therefore, we would only need to round and convert them integers in the blackbox function. That's the recommended behavior by the creators of the package.

In [5]:
#### Handling Discrete Values ####
from sklearn.ensemble import RandomForestClassifier
def black_box_function(n_estimators, max_depth, max_features):
    ## handle discrete
    n_estimators = int(round(n_estimators))
    max_depth = int(round(max_depth))
    max_features = int(round(max_features))
    ## throw an AssertionError at an earlier level if not int
    assert type(n_estimators) == int
    assert type(max_depth) == int
    assert type(max_features) == int
    #### Build Classifier ####
    classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth= max_depth,
                                        max_features=max_features, random_state=0, verbose=0, n_jobs=-1)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    return classifier.score(X_test, y_test)

pbounds = {'n_estimators': (5, 100), 
           'max_depth': (5, 25), 
           'max_features': (5, 20)}

optimizer = BayesianOptimization(f=black_box_function, pbounds=pbounds, random_state=0)
optimizer.maximize(init_points=5, n_iter=10)

print(optimizer.max)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   | max_depth | max_fe... | n_esti... |
-------------------------------------------------------------
|  1        |  0.9737   |  15.98    |  15.73    |  62.26    |
|  2        |  0.9825   |  15.9     |  11.35    |  66.36    |
|  3        |  0.9825   |  13.75    |  18.38    |  96.55    |
|  4        |  0.9737   |  12.67    |  16.88    |  55.25    |
|  5        |  0.9737   |  16.36    |  18.88    |  11.75    |
|  6        |  0.9298   |  5.0      |  5.0      |  5.0      |
|  7        |  0.9474   |  5.0      |  5.0      |  100.0    |
|  8        |  0.9649   |  25.0     |  5.0      |  100.0    |
|  9        |  0.9561   |  24.86    |  5.18     |  21.79    |
|  10       |  0.9737   |  5.007    |  19.97    |  77.48    |
|  11       |  0.9561   |  24.99    |  18.94    |  5.084    |
|  12       |  0.9561   |  5.0      |  5.0      |  41.16    |
|  13       |  0.9649   |  24.82    |  5.059    |  62.63    |
|  14       |  0.9649   |  16.2     |  5.017    |  83.09    |
|  15   

## Categorical Values

Categorical values are handled very similarly to Discrete values, nevertheless, you would only need a reference list. First, we encode the categories into integers such as in `LabelEncoder` (order is crucial here), and then the balckbox function has to unpack the right value from the reference list.

In [3]:
#### Handling Categorical values ####
from sklearn.svm import SVC
def black_box_function(C, kernel, degree, gamma):
    # handle categorical
    kernel = int(round(0.234567))
    # throw an AssertionError at an earlier level if not int
    assert type(kernel) == int
    kernel = kernels[kernel] # unpack from reference list
    
    degree = int(round(degree))
    assert type(degree) == int

    classifier = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, random_state=0, verbose=0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test) # irrelevent!
    return classifier.score(X_test, y_test)

# reference list
kernels = ['linear', 'rbf', 'sigmoid'] # poly kernel is removed, due to computational expense
# encoded list
kernels_encoded = [i for i in range(len(kernels))]

pbounds = {'C': (0, 1), 
           'kernel': (0, 2), 
           'degree': (1, 3),
           'gamma':(0, 1)}

optimizer = BayesianOptimization(f=black_box_function, pbounds=pbounds, random_state=0)
optimizer.maximize(init_points=5, n_iter=10)

print(optimizer.max)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

NameError: name 'BayesianOptimization' is not defined

## Remarks

* Interesting to see that the method doesn't need many exploitation steps, it starts to converge immediatly after 5 steps.
* We'd still need a function convert the parameters back to their true values for interpretability.